In [1]:
import os
import urllib.request
import zipfile
import copy
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [4]:
# Data augmentation and normalization for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)

# Just normalization for validation
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

# Generators
train_generator = train_datagen.flow_from_directory(
    '../../data/processed/train',
    target_size=(224, 224),
    batch_size=4,
    shuffle=True,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '../../data/processed/test',
    target_size=(224, 224),
    batch_size=4,
    shuffle=False,
    class_mode='categorical'
)

Found 12329 images belonging to 38 classes.
Found 3098 images belonging to 38 classes.


In [5]:
# Load the pre-trained model, without the top layer
base_model = ResNet50V2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet')

base_model.trainable = False

# Create a new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(38, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.legacy.SGD(lr=0.001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 38)                77862     
                                                                 
Total params: 23642662 (90.19 MB)
Trainable params: 77862 (304.15 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


/Users/jc/anaconda3/envs/ml-env/lib/python3.11/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
val_loss, val_acc = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print('Validation accuracy before transfer learning:', val_acc)


774/774 [==============================] - 92s 119ms/step - loss: 4.3290 - accuracy: 0.0245
Validation accuracy before transfer learning: 0.02454780414700508


In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

Epoch 1/10
 858/3082 [=======>......................] - ETA: 6:19 - loss: 3.5825 - accuracy: 0.1818

In [16]:
val_loss, val_acc = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print('Validation accuracy:', val_acc)

ValueError: Asked to retrieve element 0, but the Sequence has length 0